In [2]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd

# Load environment variables
load_dotenv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/.env')

# Declare API key and client info
FOURSQUARE_API_KEY = os.getenv("FOURSQUARE_API_KEY")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

# URL for places query
base_url = 'https://api.foursquare.com/v3/places/search'

# Load dataframe from CSV
montreal_bike_data = pd.read_csv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/montreal_df.csv')

# Initialize empty list to store data for DataFrame
data_for_df = []

# Parameters for API request
params = {
    'query': 'restaurant',
    'radius': 1000,
    'limit': 10,
    'sort': 'rating'
}

headers = {
    "Accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}

# Loop through dataframe to make API requests
for index, row in montreal_bike_data.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    ll_joined = f"{latitude},{longitude}"
    params['ll'] = ll_joined

    try:
        response = requests.get(base_url, params=params, headers=headers, timeout=30)
        response.raise_for_status()

        result_data = response.json()
        
        # Extract relevant information from the result_data
        for result in result_data.get('results', []):
            venue_name = result.get('name')
            address = result['location'].get('formatted_address')
            latitude = result['geocodes']['main']['latitude']
            longitude = result['geocodes']['main']['longitude']
            categories = [cat['name'] for cat in result['categories']]
            rating = result.get('rating', None)
            total_ratings = result['stats']['total_ratings'] if 'stats' in result else None
        
        #get further details to calculate a 
            
          # Append extracted data to data_for_df as a dictionary
            data_for_df.append({
                'Name': venue_name,
                'Address': address,
                'Latitude': latitude,
                'Longitude': longitude,
                'Categories': categories,
                'Rating': rating,
                'TotalRatings': total_ratings
            })
    
    except requests.exceptions.RequestException as e:
        print(f"Request failed for latitude {latitude}, longitude {longitude}. Error: {e}")

# Convert the list of dictionaries into a DataFrame
foursquare_df = pd.DataFrame(data_for_df)

# Display the DataFrame
print(foursquare_df.head())
foursquare_df.to_csv('foursquare_data.csv', index=False)

                        Name  \
0                      Cadet   
1                  L'Express   
2       Le Club Espresso Bar   
3              Bouillon Bilk   
4  Pourquoi Pas Espresso Bar   

                                             Address   Latitude  Longitude  \
0  1431 Saint-Laurent Blvd (entre De Maisonneuve ...  45.510388 -73.564365   
1  3927 Saint-Denis Rue (entre Duluth & Roy), Mon...  45.520186 -73.574269   
2          3801 Saint-Denis Rue, Montréal QC H2W 2M4  45.519481 -73.572648   
3  1595 Saint-Laurent Blvd (entre De Maisonneuve ...  45.511038 -73.565936   
4  1447 Atateken Rue (entre De Maisonneuve & Ste-...  45.517964 -73.557776   

                        Categories Rating TotalRatings  
0           [Restaurant, Wine Bar]   None         None  
1      [Bistro, French Restaurant]   None         None  
2                           [Café]   None         None  
3                     [Restaurant]   None         None  
4  [Café, Coffee Shop, Restaurant]   None         Non

In [13]:
foursquare_df.shape

(7992, 7)

In [22]:
null_rating_count = df['rating'].notnull().sum()

In [1]:
null_rating_count


NameError: name 'null_rating_count' is not defined